In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/hinditoeng/train.csv


In [5]:
!pip install evaluate
import re
import warnings
import os
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    TrainerCallback,
)
from datasets import Dataset
import evaluate
import torch
import pandas as pd
import numpy as np
import json

# Disable WANDB
os.environ["WANDB_DISABLED"] = "true"

# Suppress warnings
warnings.filterwarnings("ignore")

# Check for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device.type == "cuda":
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
else:
    print("GPU not available. Using CPU instead.")

# Load dataset
FILENAME = "/kaggle/input/hinditoeng/train.csv"  # Replace with your dataset file name
translation_data = pd.read_csv(FILENAME)
translation_data.columns

GPU not available. Using CPU instead.


Index(['Unnamed: 0', 'hindi', 'english'], dtype='object')

In [1]:
!pip install evaluate
!pip install transformers datasets evaluate torch
!pip install sacrebleu
import re
import warnings
import os
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    TrainerCallback,
)
from datasets import Dataset
import evaluate
import torch
import pandas as pd
import numpy as np
import json

# Disable WANDB
os.environ["WANDB_DISABLED"] = "true"

# Suppress warnings
warnings.filterwarnings("ignore")

# Check for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device.type == "cuda":
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
else:
    print("GPU not available. Using CPU instead.")

# Load dataset
FILENAME = "/kaggle/input/hinditoeng/train.csv"  # Replace with your dataset file name
translation_data = pd.read_csv(FILENAME)

# Rename columns for consistency
translation_data.rename(columns={"english_sentence": "english", "hindi_sentence": "hindi"}, inplace=True)

# Preprocessing function
def preprocess_text(text):
    if not isinstance(text, str):
        text = str(text)
    text = re.sub(r"[^a-zA-Z\u0900-\u097F\s]", "", text)  # Keep English and Hindi characters
    return text.strip()

# Preprocess text
translation_data["english"] = translation_data["english"].apply(preprocess_text)
translation_data["hindi"] = translation_data["hindi"].apply(preprocess_text)

# Split dataset into train and test
train_size = 0.8
train_data = translation_data.sample(frac=train_size, random_state=42)
test_data = translation_data.drop(train_data.index)

# Convert to Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_data)
test_dataset = Dataset.from_pandas(test_data)

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("t5-small")
model = AutoModelForSeq2SeqLM.from_pretrained("t5-small").to(device)

# Tokenize function
def tokenize_function(examples):
    model_inputs = tokenizer(
        examples["english"], max_length=64, truncation=True, padding="max_length"
    )
    labels = tokenizer(
        examples["hindi"], max_length=64, truncation=True, padding="max_length"
    ).input_ids
    model_inputs["labels"] = labels
    return model_inputs

# Tokenize datasets
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Set dataset format for PyTorch
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

# Training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results_english_hindi",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,  # Increased for GPU
    per_device_eval_batch_size=16,  # Increased for GPU
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=10,
    predict_with_generate=True,
    logging_dir="./logs_english_hindi",
    logging_steps=10,
    gradient_accumulation_steps=2,  # Accumulate gradients for larger effective batch size
    fp16=True,  # Enable mixed precision
    dataloader_num_workers=2,
    remove_unused_columns=False,
    save_strategy="epoch",
)

# Evaluation metric (BLEU)
metric = evaluate.load("sacrebleu")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    decoded_labels = [[label] for label in decoded_labels]  # BLEU expects nested lists
    bleu_result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    accuracy = np.mean(
        [pred.strip() == ref[0].strip() for pred, ref in zip(decoded_preds, decoded_labels)]
    )
    return {"bleu": bleu_result["score"], "accuracy": accuracy}

# Custom logging callback
class CustomTrainerCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs:
            print(f"Epoch: {state.epoch}, Logs: {logs}")

# Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    callbacks=[CustomTrainerCallback()],
)

# Train the model
print("Training the model on the English-Hindi dataset...")
train_result = trainer.train()

# Save training history
history_file = "./results_english_hindi/training_history.json"
with open(history_file, "w") as f:
    json.dump(train_result.metrics, f)
print(f"Training history saved to {history_file}")

# Save the model
model.save_pretrained("./results_english_hindi/model")
tokenizer.save_pretrained("./results_english_hindi/tokenizer")
print("Model and tokenizer saved to './results_english_hindi/'")

# Evaluate the model
results = trainer.evaluate()
print("Evaluation Results:", results)

# Save evaluation results
evaluation_file = "./results_english_hindi/evaluation_results.json"
with open(evaluation_file, "w") as f:
    json.dump(results, f)
print(f"Evaluation results saved to {evaluation_file}")

# Translate function
def translate_english_to_hindi(english_sentence):
    inputs = tokenizer(
        english_sentence, return_tensors="pt", padding=True, truncation=True, max_length=64
    ).to(device)
    outputs = model.generate(inputs.input_ids, max_length=128, num_beams=4, early_stopping=True)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Example translation
english_input = "This is a test sentence for translation."
hindi_translation = translate_english_to_hindi(english_input)
print("English:", english_input)
print("Hindi:", hindi_translation)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 6.6 MB/s eta 0:00:00
Using GPU: Tesla P100-PCIE-16GB


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

...

Epoch: 9.001367989056087, Logs: {'loss': 0.036, 'grad_norm': 0.08481262624263763, 'learning_rate': 2.001563721657545e-06, 'epoch': 9.001367989056087}
Epoch: 9.005276529216337, Logs: {'loss': 0.0417, 'grad_norm': 0.10203515738248825, 'learning_rate': 1.9937451133698206e-06, 'epoch': 9.005276529216337}
Epoch: 9.009185069376588, Logs: {'loss': 0.0394, 'grad_norm': 0.1734808087348938, 'learning_rate': 1.9859265050820955e-06, 'epoch': 9.009185069376588}
Epoch: 9.013093609536838, Logs: {'loss': 0.0424, 'grad_norm': 0.3041340708732605, 'learning_rate': 1.978107896794371e-06, 'epoch': 9.013093609536838}
Epoch: 9.017002149697088, Logs: {'loss': 0.0344, 'grad_norm': 0.17183080315589905, 'learning_rate': 1.970289288506646e-06, 'epoch': 9.017002149697088}
Epoch: 9.020910689857338, Logs: {'loss': 0.0402, 'grad_norm': 0.19109117984771729, 'learning_rate': 1.962470680218921e-06, 'epoch': 9.020910689857338}
Epoch: 9.024819230017588, Logs: {'loss': 0.0369, 'grad_norm': 0.3314269483089447, 'learning_rat

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Epoch: 9.998045729919875, Logs: {'eval_loss': 0.04147132486104965, 'eval_bleu': 0.0, 'eval_accuracy': 0.9457584050039093, 'eval_runtime': 354.7961, 'eval_samples_per_second': 57.678, 'eval_steps_per_second': 3.605, 'epoch': 9.998045729919875}
Epoch: 9.998045729919875, Logs: {'train_runtime': 8479.2766, 'train_samples_per_second': 96.539, 'train_steps_per_second': 3.017, 'total_flos': 1.3846041493241856e+16, 'train_loss': 0.06780480503150502, 'epoch': 9.998045729919875}
Training history saved to ./results_english_hindi/training_history.json
Model and tokenizer saved to './results_english_hindi/'


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Epoch: 9.998045729919875, Logs: {'eval_loss': 0.04147132486104965, 'eval_bleu': 0.0, 'eval_accuracy': 0.9457584050039093, 'eval_runtime': 346.4635, 'eval_samples_per_second': 59.065, 'eval_steps_per_second': 3.692, 'epoch': 9.998045729919875}
Evaluation Results: {'eval_loss': 0.04147132486104965, 'eval_bleu': 0.0, 'eval_accuracy': 0.9457584050039093, 'eval_runtime': 346.4635, 'eval_samples_per_second': 59.065, 'eval_steps_per_second': 3.692, 'epoch': 9.998045729919875}
Evaluation results saved to ./results_english_hindi/evaluation_results.json
English: This is a test sentence for translation.
Hindi:    
